In [1]:
import urllib.request, json 
import requests
import pandas as pd
import numpy as np
from scipy.stats import linregress
import psycopg2

In [2]:
try:
    connect_str = "dbname='d831qh80eusrfh' user='nsuzweabkeyhdi' host='ec2-54-247-117-145.eu-west-1.compute.amazonaws.com' " + \
                  "password='db6fda2fb0ad1826fbc32e0921a789ca0b1b9a6fc52d59a16b11571c6bb69674'"
    # use our connection values to establish a connection
    conn = psycopg2.connect(connect_str)
    # create a psycopg2 cursor that can execute queries
    cursor = conn.cursor()
    # run a SELECT statement - no data in there, but we can try it
    cursor.execute("""select experiment_id,qpcr_plate_id,(case when col_num='01' or col_num='02'  then 'a'
		   when col_num='03' or col_num='04'  then 'b'
		   when col_num='05' or col_num='06'  then 'c'
		   when col_num='07' or col_num='08'  then 'd'
		   when col_num='09' or col_num='10'  then 'e'
		   when col_num='11' or col_num='12'  then 'f'
		  end) as colgrp,ARRAY_AGG(DISTINCT qpcr_well) as wells from (select *,substring(qpcr_well,2,2) as col_num from app_qpcrresultsmodel) as temp group by experiment_id,qpcr_plate_id,colgrp""")
    well_by_id_assay = cursor.fetchall()
except Exception as e:
    print("Uh oh, can't connect. Invalid dbname, user or password?")
    print(e)

In [3]:
rows = ['NTC','50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','1000.0 cp ',
         '50.0 ng HgDNA ','100.0 cp ','10000.0 cp ','500.0 ng HgDNA ']
ind = ['Expt','qPCR Plate','Reps']
dep = ['ID Template Name','#Ct Pos','#Ct Valid', '#Tm Specif', '#Tm PD', '#Tm NS', 'Mean Ct', 'Min Ct',
       'Max Ct', 'Min Tm1', 'Max Tm1','Specif ng/ul Pos', 'Mean Specif ng/ul','Mean NS ng/ul', 'Mean PD ng/ul'] 

In [4]:
def list_to_string(value):
    if not isinstance(value,float):
        return "".join(str(e) for e in value)
    return None

In [5]:
def dataframe_list_to_string(df):
    list_cols = ['ID Assay Conc.','ID Assay Name','ID Human Conc.','ID Human Name',
               'ID Template Conc.','ID Template Name','PA Assay Conc.','PA Assay Name',
               'PA Human Conc.','PA Human Name','PA Template Conc.','PA Template Name']
    for column in list_cols:
        if column in df.columns:
            df[column] = df[column].apply(list_to_string)
    return df

In [6]:
def find_sample(row):    
    name = ''
    if row['ID Template Conc.']:
        name += '%s cp '%row['ID Template Conc.']
    if row['ID Human Conc.']:
        name += '%s ng HgDNA '%row['ID Human Conc.']
    if 'PA Template Conc.'in row and row['PA Template Conc.']:
        name += 'PA %s cp '%row['PA Template Conc.']
    if 'PA Human Conc.' in row and row['PA Human Conc.']:
        if 'PA' not in name:
            name += 'PA %s ng HgDNA '%row['PA Human Conc.']
        else:
            name += '%s ng HgDNA '%row['PA Human Conc.']
    if name =='':
        name = 'NTC'
    return name
    

In [7]:
def get_master_summary_data(experiments,well_by_id_assay):    
    summary_table = []
    for (experiment_id,qpcr_plate_id,grp,wells) in well_by_id_assay:     
        
        if experiment_id in experiments:
            try:
                _url = "https://assay-screening.herokuapp.com/api/well-results/?expt={}&plate_id={}&wells={}".format(experiment_id,qpcr_plate_id,wells)
                response =json.loads(requests.get(_url).text)                
                summary_df = dataframe_list_to_string(pd.DataFrame(response['summary_table']))
                summary_df['sample'] = summary_df.apply(find_sample,axis=1)
                summary_by_sample=get_summary_by_sample(summary_df,rows,ind,dep)
                summary_table.append(summary_by_sample)
            except:
                print('Failed to extract wells {} from plate {}'.format(wells,qpcr_plate_id))
    summary_with_eff_r2 = calculate_eff_r2(summary_table,rows)
    summary_with_eff_r2 = summary_with_eff_r2.sort_index(axis=1, level=[0, 1])
    
    return summary_with_eff_r2

In [9]:
def calc_eff(row):
    try:                
        expected_cols = ['50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','100.0 cp ','1000.0 cp ','10000.0 cp ']
        cols = list(filter(lambda x : (x,'Mean Ct') in row.index ,expected_cols))        
        idx = pd.IndexSlice
        y = np.array([float(n) for n in row.loc[idx[cols, 'Mean Ct']].values])
        x = np.log10([float(n.split()[0]) for n in cols])               
        mask = ~np.isnan(x) & ~np.isnan(y)        
        print(x[mask],y[mask])
        lin_fit = linregress(x[mask], y[mask])
        eff = (10**(-1 / lin_fit.slope) - 1) * 100
    except ValueError as ve:        
        eff = 0
        
    return eff

def calc_r2(row):
    try:        
        expected_cols = ['50.0 cp ','5000.0 cp ','500.0 cp ','10.0 cp ','100.0 cp ','1000.0 cp ','10000.0 cp ']
        cols = list(filter(lambda x : (x,'Mean Ct') in row.index ,expected_cols))
        idx = pd.IndexSlice
        row = row.sort_index()
        y = np.array([float(n) for n in row.loc[idx[cols, 'Mean Ct']].values])
        x = np.log10([float(n.split()[0]) for n in cols])
        
        mask = ~np.isnan(x) & ~np.isnan(y)
        
        slope, intercept, r_value, p_value, std_err = linregress(x[mask], y[mask])
        r2 = r_value**2
    except ValueError as ve:
        r2 = np.nan
        
    return r2

def removeErrFrames(frames):
    expt_list=[]
    new_df=frames[0].T
    new_df=new_df.drop(new_df.index[0])
    for i in range(1,len(frames)):           
        new_df=new_df.append(frames[i].T)        
        
    for index in sorted(expt_list, reverse=True):
        print(frames[index])
        del frames[index]
    return frames
    

In [10]:
def get_summary_by_sample(df,rows,ind,dep):
    idx = pd.IndexSlice
#     df = df[df['qPCR Plate']!='A81_E214_1_ID'] 
    frames = []    
# ['Expt','plate_id','Group','templates']
    for n, grp in df.groupby(['qPCR Plate','ID Assay Name']):    
        a = pd.DataFrame(grp.set_index('sample').loc[rows].stack())
        a = a.sort_index(level=[0, 1])
        a = a.sort_index()
        b = a.loc[idx['NTC', ind], :]
        b.index = pd.MultiIndex.from_product([['Common'], b.index.get_level_values(1)])
        c = a.loc[idx[:, dep], :]

        df = pd.concat([b, c])
        df.columns = pd.MultiIndex.from_tuples([n])
        #df = df.rename(columns={0: n[2]})
        frames.append(df)
    return frames[0]

In [11]:
def calculate_eff_r2(df,rows):
    frames=removeErrFrames(df)
    frames_df = pd.concat(frames, axis=1).reindex(rows, level=0)
    frames_df = frames_df.sort_index().T
    frames_df[('Common', 'Efficiency')] = frames_df.apply(calc_eff, axis=1)
    frames_df[('Common', 'R Squared')] = frames_df.apply(calc_r2,axis=1)
    return frames_df

In [12]:
experiments = ['A81_E307']
summary_table = get_master_summary_data(experiments,well_by_id_assay)

c:\users\prasanna.gnanaraj\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


[1. 2. 3. 4.] [33.17765236 29.18826866 24.88851166 22.94439507]
[1. 2. 3. 4.] [32.9611969  30.52986336 26.15118504 24.49346066]
[1.] [32.99442577]
[1. 2. 3. 4.] [33.98553022 28.48408413 24.66755962 22.51852417]
[1. 2. 3. 4.] [33.24439144 29.63673401 25.81524563 23.25252247]
[1. 2. 3. 4.] [34.2562472  30.56502533 26.5171442  23.69455147]
[1. 2. 3. 4.] [33.29965687 30.48225307 27.29253006 22.8167429 ]
[1. 2. 3. 4.] [22.79812098 22.69056988 22.47411633 20.92079926]
[1. 2. 3. 4.] [31.08098984 29.50765133 26.96467972 22.26776409]
[1. 2. 3. 4.] [26.82554626 27.23434448 25.81972504 21.91572762]
[1. 2. 3. 4.] [31.80114222 30.34225368 27.03196239 22.7941618 ]
[1. 2. 3. 4.] [29.2271924  28.64180946 25.53065491 21.40113831]
[1. 2. 3. 4.] [36.44516468 32.47130585 28.7198801  24.92990017]
[1. 2. 3. 4.] [23.76252794 23.86692238 23.18078613 20.35411739]
[1. 2. 3. 4.] [29.60605812 29.49693108 25.90547371 22.30356312]
[1. 2. 3. 4.] [25.01423979 24.44539833 24.70764828 21.14583397]
[1. 2. 3. 4.] [33.697

c:\users\prasanna.gnanaraj\appdata\local\programs\python\python36\lib\site-packages\scipy\stats\_stats_mstats_common.py:107: RuntimeWarning: invalid value encountered in double_scalars
  slope = r_num / ssxm
c:\users\prasanna.gnanaraj\appdata\local\programs\python\python36\lib\site-packages\scipy\stats\_stats_mstats_common.py:117: RuntimeWarning: invalid value encountered in sqrt
  t = r * np.sqrt(df / ((1.0 - r + TINY)*(1.0 + r + TINY)))
c:\users\prasanna.gnanaraj\appdata\local\programs\python\python36\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
c:\users\prasanna.gnanaraj\appdata\local\programs\python\python36\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
c:\users\prasanna.gnanaraj\appdata\local\programs\python\python36\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: R

In [13]:
summary_table.to_csv('group_by_columns.csv')

In [16]:
(10**(-1 / linregress([1,2, 3, 4], [33.17765236,29.18826866,24.88851166,22.94439507]).slope) - 1) * 100

93.0714826769139

In [17]:
summary_table

10.0 cp                           \
                                     #Ct Pos #Ct Valid #Tm NS #Tm PD   
A81_E307_1 AMR_NDM_7.x_NDM25_NDM30     1 | 4     4 | 4  2 | 4  1 | 4   
           AMR_NDM_10.x_NDM31_NDM39    2 | 4     2 | 4  2 | 4  2 | 4   
           AMR_NDM_12.x_NDM32_NDM39    0 | 4     4 | 4  4 | 4  0 | 4   
           AMR_NDM_14.x_NDM40_NDM48    1 | 4     3 | 4  1 | 4  2 | 4   
           AMR_NDM_16.x_NDM41_NDM47    0 | 4     4 | 4  1 | 4  0 | 4   
           AMR_NDM_18.x_NDM41_NDM49    0 | 4     3 | 4  0 | 4  1 | 4   
A81_E307_2 AMR_NDM_x.5_NDM27_NDM29     0 | 4     4 | 4  1 | 4  1 | 4   
           AMR_NDM_x.6_NDM33_NDM35     0 | 4     4 | 4  0 | 4  4 | 4   
           AMR_NDM_x.7_NDM33_NDM36     1 | 4     4 | 4  0 | 4  2 | 4   
           AMR_NDM_x.8_NDM33_NDM37     0 | 4     4 | 4  0 | 4  4 | 4   
           AMR_NDM_x.9_NDM42_NDM45     1 | 4     4 | 4  1 | 4  2 | 4   
           AMR_NDM_x.10_NDM42_NDM46    1 | 4     4 | 4  4 | 4  0 | 4   
A81_E307_4 AMR_NDM_x.5_NDM27_NDM29     1 | 4     4 | 4  1 | 4  1 | 4   
           AMR_NDM_x.6_NDM33_NDM35     0 | 4     4 | 4  0 | 4  4 | 4   
           AMR_NDM_x.7_NDM33_NDM36     2 | 4     4 | 4  0 | 4  4 | 4   
           AMR_NDM_x.8_NDM33_NDM37     0 | 4     4 | 4  1 | 4  4 | 4   
           AMR_NDM_x.9_NDM42_NDM45     4 | 4     4 | 4  0 | 4  0 | 4   
           AMR_NDM_x.10_NDM42_NDM46    0 | 4     4 | 4  0 | 4  1 | 4   

                                                                       \
                                    #Tm Specif       ID Template Name   
A81_E307_1 AMR_NDM_7.x_NDM25_NDM30       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_10.x_NDM31_NDM39      2 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_12.x_NDM32_NDM39      0 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_14.x_NDM40_NDM48      2 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_16.x_NDM41_NDM47      4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_18.x_NDM41_NDM49      3 | 4  K.pneumoniae-NDM_2146   
A81_E307_2 AMR_NDM_x.5_NDM27_NDM29       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.6_NDM33_NDM35       0 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.7_NDM33_NDM36       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.8_NDM33_NDM37       2 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.9_NDM42_NDM45       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.10_NDM42_NDM46      2 | 4  K.pneumoniae-NDM_2146   
A81_E307_4 AMR_NDM_x.5_NDM27_NDM29       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.6_NDM33_NDM35       0 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.7_NDM33_NDM36       2 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.8_NDM33_NDM37       1 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.9_NDM42_NDM45       4 | 4  K.pneumoniae-NDM_2146   
           AMR_NDM_x.10_NDM42_NDM46      4 | 4  K.pneumoniae-NDM_2146   

                                                                              \
                                      Max Ct  Max Tm1  Mean Ct Mean NS ng/ul   
A81_E307_1 AMR_NDM_7.x_NDM25_NDM30    33.757  92.5808  33.1777             0   
           AMR_NDM_10.x_NDM31_NDM39  33.1984  92.9915  32.9612             0   
           AMR_NDM_12.x_NDM32_NDM39  34.9095  93.1284  32.9944             0   
           AMR_NDM_14.x_NDM40_NDM48  36.9837   92.307  33.9855             0   
           AMR_NDM_16.x_NDM41_NDM47  34.9903  92.5808  33.2444             0   
           AMR_NDM_18.x_NDM41_NDM49  35.9324  92.4439  34.2562             0   
A81_E307_2 AMR_NDM_x.5_NDM27_NDM29   34.1779  89.9899  33.2997             0   
           AMR_NDM_x.6_NDM33_NDM35   23.0356  77.1185  22.7981             0   
           AMR_NDM_x.7_NDM33_NDM36   32.4993  92.8655   31.081             0   
           AMR_NDM_x.8_NDM33_NDM37    28.642  92.8655  26.8255             0   
           AMR_NDM_x.9_NDM42_NDM45   34.3036  92.3177  31.8011             0   
           AMR_NDM_x.10_NDM42_NDM46  31.9374  92.3177  29.2272      0.644427   
A81_E307_4 AMR